In [2]:
import os

In [3]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

#need to change according to my training needs
@dataclass(frozen=True)
class ModelInferenceConfig:
    model_path: Path
    all_params: dict


In [7]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelInferenceConfig:
        config = self.config.model_training
        params = self.params.HeartDiseaseNN

        model_training_config = ModelInferenceConfig(
            root_dir=config.root_dir,
            model_path = config.model_path,
            input_dim = params.input_dim,
            hidden1_dim = params.hidden1_dim,
            hidden2_dim = params.hidden2_dim,
            output_dim = params.output_dim
        )
        
        return model_training_config

In [ ]:
from src.mlops_project.utils.model_architecture import NeuralNetwork
import torch

In [ ]:
class ModelInference:
    def __init__(self, config: ModelInferenceConfig) -> None:
        self.config=config
    
    def predict(self, input):

        model = NeuralNetwork(self.config.input_dim, 
                              self.config.hidden1_dim, 
                              self.config.hidden2_dim, 
                              self.config.output_dim)
        
        model.load_state_dict(torch.load("model_path.pth"))

        input_tensor = torch.tensor(input, dtype=torch.float32)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted_class = torch.max(output, dim=1)

        return predicted_class
        